In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/clean_raw_data.csv
/kaggle/input/clean_data_mean.csv
/kaggle/input/raw_data.csv
/kaggle/input/clean_data_GCN.csv
/kaggle/input/FillZero_minmax_baseline/clean_data_minmax_fill-zero.csv
/kaggle/input/FillZero_minmax_baseline/clean_week2/val/val_week1_2.csv
/kaggle/input/FillZero_minmax_baseline/clean_week2/test/test_week2.csv
/kaggle/input/FillZero_minmax_baseline/clean_week2/train/clean_data_week2.csv
/kaggle/input/FillZero_minmax_baseline/clean_week2/train/5-folds/data_part_2.csv
/kaggle/input/FillZero_minmax_baseline/clean_week2/train/5-folds/data_part_3.csv
/kaggle/input/FillZero_minmax_baseline/clean_week2/train/5-folds/data_part_4.csv
/kaggle/input/FillZero_minmax_baseline/clean_week2/train/5-folds/data_part_1.csv
/kaggle/input/FillZero_minmax_baseline/clean_week2/train/5-folds/data_part_5.csv
/kaggle/input/FillZero_minmax_baseline/clean_week3/val/val_week1_2_3.csv
/kaggle/input/FillZero_minmax_baseline/clean_week3/test/test_week3.csv
/kaggle/input/FillZero_minmax_bas

In [2]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import backend as K
from keras_tuner import RandomSearch
from sklearn.model_selection import StratifiedKFold
import time
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_recall_fscore_support, roc_auc_score

In [3]:
# Biến global cho base path
BASE_PATH = "/kaggle/input/Mean_minmax_baseline"
# Tuần và số phần fold
weeks = ['week1', 'week2', 'week3', 'week4']
fold_parts = 5

# Tạo five_fold_files
five_fold_files = {
    week: [
        f"{BASE_PATH}/clean_{week}/train/5-folds/data_part_{i}.csv"
        for i in range(1, fold_parts + 1)
    ]
    for week in weeks
}

# Tạo file_validation
file_validation = {
    'week1': [f"{BASE_PATH}/clean_week1/val/val_week1.csv"],
    'week2': [f"{BASE_PATH}/clean_week2/val/val_week1_2.csv"],
    'week3': [f"{BASE_PATH}/clean_week3/val/val_week1_2_3.csv"],
    'week4': [f"{BASE_PATH}/clean_week4/val/val_week1_2_3_4.csv"]
}

# Tạo file_test
file_test = {
    week: [f"{BASE_PATH}/clean_{week}/test/test_{week}.csv"]
    for week in weeks
}


## Tìm siêu tham số tốt nhất cho từng tuần

In [4]:
# Định nghĩa Focal Loss
def focal_loss(gamma=2., alpha=0.25):
    def focal_loss_fixed(y_true, y_pred):
        y_pred = K.clip(y_pred, K.epsilon(), 1. - K.epsilon())
        cross_entropy = -y_true * K.log(y_pred)
        loss = alpha * K.pow(1 - y_pred, gamma) * cross_entropy
        return K.sum(loss, axis=-1)
    return focal_loss_fixed

# Tạo hàm train cho từng tuần
def train_week_model(week_number, file_paths_train, file_validataion):
    # Đọc dữ liệu
    train_data = pd.read_csv(file_paths_train)
    val_data = pd.read_csv(file_validataion)
    
    # Tách đặc trưng và nhãn
    X_train = train_data.drop(columns=["classification_encoded", "user_id", "course_id", "school", "enroll_time", "classification"])
    y_train = train_data["classification_encoded"]

    X_val = val_data.drop(columns=["classification_encoded", "user_id", "course_id", "school", "enroll_time", "classification"])
    y_val = val_data["classification_encoded"]
    
    # Áp dụng Over-sampling cho dữ liệu huấn luyện bằng SMOTE
    oversampler = SMOTE(sampling_strategy='auto', random_state=42)
    X_train_res, y_train_res = oversampler.fit_resample(X_train, y_train)
    
    # Reshape dữ liệu cho mô hình BiLSTM
    X_train_res = X_train_res.values.reshape(X_train_res.shape[0], X_train_res.shape[1], 1)
    X_val = X_val.values.reshape(X_val.shape[0], X_val.shape[1], 1)
    
    # One-hot encode nhãn
    y_train_res = tf.keras.utils.to_categorical(y_train_res, num_classes=5)
    y_val = tf.keras.utils.to_categorical(y_val, num_classes=5)
    
    def build_model(hp):
        inputs = tf.keras.Input(shape=(X_train_res.shape[1], 1))  # Khởi tạo đầu vào
        
        # GRU layer 1
        x = layers.GRU(
            units=hp.Int('units_1', min_value=32, max_value=256, step=32),
            return_sequences=True
        )(inputs)
        x = layers.Dropout(rate=hp.Float('dropout_1', min_value=0.1, max_value=0.5, step=0.1))(x)
        
        # GRU layer 2
        x = layers.GRU(
            units=hp.Int('units_2', min_value=32, max_value=256, step=32),
            return_sequences=False
        )(x)
        x = layers.Dropout(rate=hp.Float('dropout_2', min_value=0.1, max_value=0.5, step=0.1))(x)
        
        # Lớp đầu ra
        outputs = layers.Dense(5, activation='softmax')(x)
        
        # Khởi tạo mô hình
        model = tf.keras.Model(inputs=inputs, outputs=outputs)
        
        # Compile với Focal Loss
        model.compile(optimizer=tf.keras.optimizers.Adam(
                          learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')),
                      loss=focal_loss(gamma=2., alpha=0.25),
                      metrics=['accuracy'])
        
        return model

    
    # Khởi tạo RandomSearch tuner
    tuner = RandomSearch(
        build_model,
        objective='val_accuracy',
        max_trials=10,
        executions_per_trial=1,
        directory='my_dir',
        project_name=f'bilstm_tuning_week{week_number}'
    )
    
    # Tìm kiếm siêu tham số tốt nhất
    tuner.search(X_train_res, y_train_res,
                 epochs=20,
                 validation_data=(X_val, y_val),
                 batch_size=32)
    
    # Trả về kết quả tối ưu cho tuần
    best_params = tuner.get_best_hyperparameters(num_trials=1)[0]
    return best_params

In [5]:
# Định nghĩa đường dẫn đến dữ liệu cho từng tuần
file_paths_train = {
    week: f"{BASE_PATH}/clean_{week}/train/clean_data_{week}.csv"
    for week in weeks
}

# Định nghĩa file_validation theo quy luật riêng
file_validation = {
    f"week{idx + 1}": f"{BASE_PATH}/clean_week{idx + 1}/val/val_week{'_'.join(str(i) for i in range(1, idx + 2))}.csv"
    for idx in range(len(weeks))
}

In [6]:
# Tìm tham số tốt nhất cho từng tuần
best_params_week1 = train_week_model(1, file_paths_train["week1"], file_validation["week1"])
best_params_week2 = train_week_model(2, file_paths_train["week2"], file_validation["week2"])
best_params_week3 = train_week_model(3, file_paths_train["week3"], file_validation["week3"])
best_params_week4 = train_week_model(4, file_paths_train["week4"], file_validation["week4"])

# In thông tin chi tiết các tham số tối ưu
print("Best Parameters for Week 1:")
for param_name in best_params_week1.values.keys():
    print(f"{param_name}: {best_params_week1.get(param_name)}")

print("\nBest Parameters for Week 2:")
for param_name in best_params_week2.values.keys():
    print(f"{param_name}: {best_params_week2.get(param_name)}")

print("\nBest Parameters for Week 3:")
for param_name in best_params_week3.values.keys():
    print(f"{param_name}: {best_params_week3.get(param_name)}")

print("\nBest Parameters for Week 4:")
for param_name in best_params_week4.values.keys():
    print(f"{param_name}: {best_params_week4.get(param_name)}")


Trial 10 Complete [00h 02m 34s]
val_accuracy: 0.9975594878196716

Best val_accuracy So Far: 0.9993898868560791
Total elapsed time: 00h 25m 31s
Best Parameters for Week 1:
units_1: 256
dropout_1: 0.2
units_2: 192
dropout_2: 0.2
learning_rate: 0.0012782268636310837

Best Parameters for Week 2:
units_1: 224
dropout_1: 0.1
units_2: 128
dropout_2: 0.1
learning_rate: 0.0009863543791958332

Best Parameters for Week 3:
units_1: 64
dropout_1: 0.30000000000000004
units_2: 128
dropout_2: 0.4
learning_rate: 0.002613956776855358

Best Parameters for Week 4:
units_1: 64
dropout_1: 0.1
units_2: 160
dropout_2: 0.4
learning_rate: 0.00033847378479185606


## Danh sách tham số tốt nhất của từng tuần

In [7]:
# Danh sách tham số tốt nhất
best_params = {
    "week1": best_params_week1,
    "week2": best_params_week2,
    "week3": best_params_week3,
    "week4": best_params_week4
}

In [8]:
from tensorflow.keras import layers
import tensorflow as tf
from tensorflow.keras import backend as K

# Định nghĩa Focal Loss
def focal_loss(gamma=2., alpha=0.25):
    def focal_loss_fixed(y_true, y_pred):
        y_pred = K.clip(y_pred, K.epsilon(), 1. - K.epsilon())
        cross_entropy = -y_true * K.log(y_pred)
        loss = alpha * K.pow(1 - y_pred, gamma) * cross_entropy
        return K.sum(loss, axis=-1)
    
    return focal_loss_fixed

# Xây dựng mô hình BiLSTM
def build_GRU_model(params, input_shape):
    inputs = tf.keras.Input(shape=input_shape)  # Định nghĩa đầu vào
    
    # GRU layer 1
    x = layers.GRU(
        units=params.get('units_1'),
        return_sequences=True
    )(inputs)
    x = layers.Dropout(rate=params.get('dropout_1', 0.2))(x)
    
    # GRU layer 2
    x = layers.GRU(
        units=params.get('units_2', 32),
        return_sequences=False
    )(x)
    x = layers.Dropout(rate=params.get('dropout_2', 0.2))(x)
    
    # Lớp đầu ra
    outputs = layers.Dense(5, activation='softmax')(x)
    
    # Khởi tạo mô hình
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    
    # Compile với Focal Loss
    model.compile(optimizer=tf.keras.optimizers.Adam(
                      learning_rate=params['learning_rate']),
                  loss=focal_loss(gamma=params.get('gamma', 2.), alpha=params.get('alpha', 0.25)),
                  metrics=['accuracy'])
    
    return model


In [9]:
# Biến lưu kết quả tổng quát
overall_results_5folds = []

# Lặp qua từng tuần
for week, file_paths in five_fold_files.items():
    print(f"\nProcessing {week} with best parameters...")
    params = best_params[week].values
    print(f"best parameters for {week}: {params}")
    
    # Biến lưu kết quả cho từng tuần
    week_results = {
        "week": week,
        "accuracy_per_fold": [],
        "precision_per_label": [],
        "recall_per_label": [],
        "f1_score_per_label": [],
        "auc_roc_per_label": [],    # AUC từng lớp
        "auc_roc_macro": [],        # AUC macro
        "auc_roc_weighted": [],     # AUC weighted (tự tính)
        "precision_macro": [],
        "recall_macro": [],
        "f1_macro": [],
        "precision_weighted": [],
        "recall_weighted": [],
        "f1_weighted": [],
        "confusion_matrices": [],
        "train_times": [],
        "test_times": []
    }

    # Lặp qua từng fold
    for i in range(len(file_paths)):
        print(f"Fold {i+1}: Using file {file_paths[i]} as test set")
        
        # Tải dữ liệu
        test_data = pd.read_csv(file_paths[i])
        train_data = pd.concat([pd.read_csv(file_paths[j]) for j in range(len(file_paths)) if j != i])
        
        # Tách X và y
        X_train = train_data.drop(columns=["classification_encoded", "user_id",
                                           "course_id", "school", "enroll_time", "classification"])
        y_train = to_categorical(train_data['classification_encoded'], num_classes=5)
        
        X_test = test_data.drop(columns=["classification_encoded", "user_id",
                                         "course_id", "school", "enroll_time", "classification"])
        y_test = to_categorical(test_data['classification_encoded'], num_classes=5)

        # Reshape dữ liệu cho LSTM
        X_train = X_train.to_numpy().reshape((X_train.shape[0], 1, X_train.shape[1]))
        X_test = X_test.to_numpy().reshape((X_test.shape[0], 1, X_test.shape[1]))

        # Xây dựng mô hình với tham số tốt nhất
        input_shape = (X_train.shape[1], X_train.shape[2])
        model = build_GRU_model(params, input_shape)
        
        # Bắt đầu tính thời gian huấn luyện
        start_train = time.time()
        model.fit(X_train, y_train, epochs=50, validation_data=(X_test, y_test), batch_size=32)
        end_train = time.time()
        
        # Bắt đầu tính thời gian kiểm thử
        start_test = time.time()
        y_pred = model.predict(X_test)
        end_test = time.time()
        
        # Tính thời gian và lưu lại
        train_time = end_train - start_train
        test_time = end_test - start_test
        week_results["train_times"].append(train_time)
        week_results["test_times"].append(test_time)

        # Đánh giá mô hình trên tập kiểm thử của fold hiện tại
        _, accuracy = model.evaluate(X_test, y_test, verbose=0)
        week_results["accuracy_per_fold"].append(accuracy)
        
        # Dự đoán
        y_pred_classes = y_pred.argmax(axis=1)
        y_test_classes = y_test.argmax(axis=1)
        
        # Tính các chỉ số cho mỗi fold
        precision, recall, f1, _ = precision_recall_fscore_support(y_test_classes, y_pred_classes, average=None)
        precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(y_test_classes, y_pred_classes, average='macro')
        precision_weighted, recall_weighted, f1_weighted, _ = precision_recall_fscore_support(y_test_classes, y_pred_classes, average='weighted')
        conf_matrix = confusion_matrix(y_test_classes, y_pred_classes)
        
        # Tính AUC-ROC
        try:
            # Tính AUC macro và theo từng lớp với OvR
            auc_macro = roc_auc_score(y_test, y_pred, multi_class="ovr", average="macro")
            auc_per_class = roc_auc_score(y_test, y_pred, multi_class="ovr", average=None)
            # Tính AUC weighted: tính trọng số theo số mẫu của từng lớp
            supports = np.bincount(y_test_classes, minlength=5)
            auc_weighted = np.sum(auc_per_class * supports) / np.sum(supports)
        except Exception as e:
            print(f"Lỗi khi tính AUC: {e}")
            auc_macro = np.nan
            auc_per_class = [np.nan] * 5
            auc_weighted = np.nan
            
        # Lưu kết quả của fold hiện tại
        week_results["precision_per_label"].append(precision)
        week_results["recall_per_label"].append(recall)
        week_results["f1_score_per_label"].append(f1)
        week_results["auc_roc_per_label"].append(auc_per_class)  # AUC từng lớp
        week_results["auc_roc_macro"].append(auc_macro)          # AUC macro
        week_results["auc_roc_weighted"].append(auc_weighted)      # AUC weighted
        week_results["confusion_matrices"].append(conf_matrix)
        week_results["precision_macro"].append(precision_macro)
        week_results["recall_macro"].append(recall_macro)
        week_results["f1_macro"].append(f1_macro)
        week_results["precision_weighted"].append(precision_weighted)
        week_results["recall_weighted"].append(recall_weighted)
        week_results["f1_weighted"].append(f1_weighted)

    # Tính trung bình cho từng nhãn
    average_precision_per_label = np.mean(week_results["precision_per_label"], axis=0)
    average_recall_per_label = np.nanmean(week_results["recall_per_label"], axis=0)
    average_f1_per_label = np.nanmean(week_results["f1_score_per_label"], axis=0)
    average_auc_per_label = np.nanmean(week_results["auc_roc_per_label"], axis=0)
    average_confusion_matrix = np.nanmean(week_results["confusion_matrices"], axis=0)
    average_train_time = sum(week_results["train_times"]) / len(week_results["train_times"])
    average_test_time = sum(week_results["test_times"]) / len(week_results["test_times"])
    average_accuracy = np.nanmean(week_results["accuracy_per_fold"])
    average_precision_macro = np.nanmean(week_results["precision_macro"])
    average_recall_macro = np.nanmean(week_results["recall_macro"])
    average_f1_macro = np.nanmean(week_results["f1_macro"])
    average_auc_macro = np.nanmean(week_results["auc_roc_macro"])
    average_precision_weighted = np.nanmean(week_results["precision_weighted"])
    average_recall_weighted = np.nanmean(week_results["recall_weighted"])
    average_f1_weighted = np.nanmean(week_results["f1_weighted"])
    average_auc_weighted = np.nanmean(week_results["auc_roc_weighted"])


    # Tạo DataFrame cho precision, recall, f1-score
    labels = np.unique(y_test_classes)  # Lấy nhãn từ y_test_classes
    metrics_df = pd.DataFrame({
        "Label": labels,
        "Average Precision": average_precision_per_label,
        "Average Recall": average_recall_per_label,
        "Average F1-Score": average_f1_per_label,
        "Average AUC": average_auc_per_label
    })
    
    # Tạo DataFrame cho confusion matrix
    confusion_df = pd.DataFrame(average_confusion_matrix, index=labels, columns=labels)
    # In kết quả Accuracy và Macro metrics
    print("\n=== Average Accuracy ===")
    print(f"{average_accuracy:.4f}")
    print("\n=== Average Macro Metrics ===")
    print(f"Macro Precision: {average_precision_macro:.4f}")
    print(f"Macro Recall: {average_recall_macro:.4f}")
    print(f"Macro F1-Score: {average_f1_macro:.4f}")
    print(f"Macro AUC-ROC: {average_auc_macro:.4f}")
    print("\n=== Average Weighted Metrics ===")
    print(f"Weighted Precision: {average_precision_weighted:.4f}")
    print(f"Weighted Recall: {average_recall_weighted:.4f}")
    print(f"Weighted F1-Score: {average_f1_weighted:.4f}")
    print(f"Weighted AUC-ROC: {average_auc_weighted:.4f}")
    print("\n=== Average Metrics per Label ===")
    print(metrics_df)
    print("\n=== Average Confusion Matrix ===")
    print(confusion_df)
    
    # Cập nhật kết quả cho tuần hiện tại
    week_results.update({
        "average_accuracy": average_accuracy,
        "average_precision_macro": average_precision_macro,
        "average_recall_macro": average_recall_macro,
        "average_f1_macro": average_f1_macro,
        "average_auc_macro": average_auc_macro,
        "average_precision_weighted": average_precision_weighted,
        "average_recall_weighted": average_recall_weighted,
        "average_f1_weighted": average_f1_weighted,
        "average_auc_weighted": average_auc_weighted,
        "average_metrics_df": metrics_df,
        "average_confusion_matrix": confusion_df,
        "average_train_times": average_train_time,
        "average_test_times": average_test_time,
    })
    overall_results_5folds.append(week_results)


Processing week1 with best parameters...
best parameters for week1: {'units_1': 256, 'dropout_1': 0.2, 'units_2': 192, 'dropout_2': 0.2, 'learning_rate': 0.0012782268636310837}
Fold 1: Using file /kaggle/input/Mean_minmax_baseline/clean_week1/train/5-folds/data_part_1.csv as test set
Epoch 1/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.6357 - loss: 0.1320 - val_accuracy: 0.6645 - val_loss: 0.1202
Epoch 2/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.6874 - loss: 0.1145 - val_accuracy: 0.6958 - val_loss: 0.1110
Epoch 3/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.6953 - loss: 0.1092 - val_accuracy: 0.7411 - val_loss: 0.1005
Epoch 4/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.7263 - loss: 0.1027 - val_accuracy: 0.7530 - val_loss: 0.0952
Epoch 5/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.7419 - loss: 0.0990 - val_accuracy: 0.7709 - val_loss: 0.0937
Epoch 6/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.7646 - l

## Kết quả cross validation trên 5-folds

In [10]:
# Duyệt qua các tuần trong overall_results
for week_result in overall_results_5folds:
    week = week_result["week"]
    average_train_time = np.mean(week_result["train_times"])
    average_test_time = np.mean(week_result["test_times"])
    average_metrics_df = week_result["average_metrics_df"]
    average_accuracy = np.mean(week_results["accuracy_per_fold"])
    average_confusion_matrix = week_result["average_confusion_matrix"]
    
    # In kết quả
    print(f"\n=== Results for {week} ===")
    print(f"Average Accurancy: {average_accuracy}")
    print(f"Average Train Time: {average_train_time:.4f} seconds")
    print(f"Average Test Time: {average_test_time:.4f} seconds")
    print(f"Average AUC Macro: {average_auc_macro}")
    print(f"Average AUC Weighted: {average_auc_weighted}")
    print("\nAverage Precision, Recall, F1-Score, AUC-ROC per Label:")
    print(average_metrics_df)
    print("\nAverage Confusion Matrix:")
    print(average_confusion_matrix)



=== Results for week1 ===
Average Accurancy: 0.9872649788856507
Average Train Time: 85.2935 seconds
Average Test Time: 0.4293 seconds
Average AUC Macro: 0.9993256037998768
Average AUC Weighted: 0.9997442275870663

Average Precision, Recall, F1-Score, AUC-ROC per Label:
   Label  Average Precision  Average Recall  Average F1-Score  Average AUC
0      0           0.751296        0.772667          0.761488     0.941516
1      1           0.680483        0.308490          0.391376     0.939923
2      2           0.800932        0.457398          0.580642     0.913691
3      3           0.742619        0.612511          0.669553     0.933674
4      4           0.893632        0.963333          0.927106     0.965592

Average Confusion Matrix:
       0     1     2      3       4
0  463.6   9.6   7.2   12.2   107.4
1   42.4  27.0   1.8    3.8    12.4
2   47.2   3.2  75.2    9.6    29.2
3   26.2   0.8   2.6  102.4    35.2
4   38.4   2.4   7.6   10.4  1544.8

=== Results for week2 ===
Average A

## Kiểm tra trên tập test

In [11]:
# Mảng lưu dữ liệu của các tuần
results = []

def process_week(week_num, best_params, results):
    print(f"\n=== Processing Week {week_num} ===")
    params = best_params[f"week{week_num}"].values
    # Đường dẫn tới dữ liệu tuần tương ứng
    train_path = f"{BASE_PATH}/clean_week{week_num}/train/clean_data_week{week_num}.csv"
    test_path = f"{BASE_PATH}/clean_week{week_num}/test/test_week{week_num}.csv"
    
    # Load dữ liệu
    train_data = pd.read_csv(train_path)
    test_data = pd.read_csv(test_path)
    
    # Tách X và y
    X_train = train_data.drop(columns=["classification_encoded", "user_id",
                                       "course_id", "school", "enroll_time", "classification"])
    y_train = train_data['classification_encoded']
    
    X_test = test_data.drop(columns=["classification_encoded", "user_id",
                                     "course_id", "school", "enroll_time", "classification"])
    y_test = test_data['classification_encoded']

    # Áp dụng SMOTE cho tập huấn luyện
    smote = SMOTE(random_state=42)
    X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

    # Chuyển đổi nhãn sang dạng one-hot
    y_train_resampled = to_categorical(y_train_resampled, num_classes=5)
    y_test = to_categorical(y_test, num_classes=5)

    # Reshape dữ liệu cho LSTM
    X_train_resampled = X_train_resampled.to_numpy().reshape((X_train_resampled.shape[0], 1, X_train_resampled.shape[1]))
    X_test = X_test.to_numpy().reshape((X_test.shape[0], 1, X_test.shape[1]))

    # Xây dựng mô hình với tham số tốt nhất
    input_shape = (X_train_resampled.shape[1], X_train_resampled.shape[2])
    model = build_GRU_model(params, input_shape)
    
    # Huấn luyện mô hình
    start_train = time.time()
    model.fit(X_train_resampled, y_train_resampled, epochs=50, validation_split=0.1, batch_size=32)
    end_train = time.time()
    
    # Kiểm thử mô hình
    start_test = time.time()
    y_pred = model.predict(X_test)
    end_test = time.time()
    
    # Tính thời gian huấn luyện và kiểm thử
    train_time = end_train - start_train
    test_time = end_test - start_test
    
    # Đánh giá mô hình
    y_pred_classes = y_pred.argmax(axis=1)
    y_test_classes = y_test.argmax(axis=1)
    
    # Tính các chỉ số Precision, Recall, F1 cho từng lớp và macro
    precision, recall, f1, _ = precision_recall_fscore_support(y_test_classes, y_pred_classes, average=None)
    precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(y_test_classes, y_pred_classes, average='macro')
    precision_weighted, recall_weighted, f1_weighted, _ = precision_recall_fscore_support(y_test_classes, y_pred_classes, average='weighted')
    conf_matrix = confusion_matrix(y_test_classes, y_pred_classes)
    accuracy = accuracy_score(y_test_classes, y_pred_classes)
    
    # Tính AUC-ROC (với one-vs-rest)
    try:
        auc_macro = roc_auc_score(y_test, y_pred, multi_class="ovr", average="macro")
        auc_per_class = roc_auc_score(y_test, y_pred, multi_class="ovr", average=None)
        # Tính AUC weighted tự tính theo trọng số mẫu của từng lớp
        supports = np.bincount(y_test_classes, minlength=5)
        auc_weighted = np.sum(auc_per_class * supports) / np.sum(supports)
    except Exception as e:
        print(f"Lỗi khi tính AUC: {e}")
        auc_macro = np.nan
        auc_per_class = [np.nan] * 5
        auc_weighted = np.nan

    # Lưu kết quả vào mảng
    results.append({
        "week": week_num,
        "train_time": train_time,
        "test_time": test_time,
        "precision": precision,
        "recall": recall,
        "f1_score": f1,
        "accuracy": accuracy,
        "precision_macro": precision_macro,
        "recall_macro": recall_macro,
        "f1_macro": f1_macro,
        "precision_weighted": precision_weighted,
        "recall_weighted": recall_weighted,
        "f1_weighted": f1_weighted,
        "auc_macro": auc_macro,
        "auc_weighted": auc_weighted,
        "auc_per_class": auc_per_class,
        "confusion_matrix": conf_matrix
    })
    
    # In kết quả chi tiết
    print("\n=== Precision, Recall, F1-Score per Label ===")
    print(pd.DataFrame({
        "Label": np.unique(y_test_classes),
        "Precision": precision,
        "Recall": recall,
        "F1-Score": f1
    }))

    print("\n=== Macro Averages & Accuracy ===")
    print(f"Macro Precision: {precision_macro:.4f}")
    print(f"Macro Recall: {recall_macro:.4f}")
    print(f"Macro F1-Score: {f1_macro:.4f}")
    print(f"Accuracy: {accuracy:.4f}")
    
    print("\n=== Weighted Averages ===")
    print(f"Weighted Precision: {precision_weighted:.4f}")
    print(f"Weighted Recall: {recall_weighted:.4f}")
    print(f"Weighted F1-Score: {f1_weighted:.4f}")
    
    print("\n=== AUC-ROC ===")
    print(f"AUC Macro: {auc_macro:.4f}")
    print(f"AUC Weighted: {auc_weighted:.4f}")
    print(f"AUC per Label: {auc_per_class}")
    
    print("\n=== Confusion Matrix ===")
    print(pd.DataFrame(conf_matrix, index=np.unique(y_test_classes), columns=np.unique(y_test_classes)))
    
    print(f"\nTrain Time: {train_time:.2f} seconds")
    print(f"Test Time: {test_time:.2f} seconds")

In [12]:
process_week(1, best_params, results)


=== Processing Week 1 ===
Epoch 1/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 7s 5ms/step - accuracy: 0.4865 - loss: 0.1934 - val_accuracy: 0.7047 - val_loss: 0.2061
Epoch 2/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accuracy: 0.5986 - loss: 0.1564 - val_accuracy: 0.6398 - val_loss: 0.1935
Epoch 3/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.6315 - loss: 0.1417 - val_accuracy: 0.6338 - val_loss: 0.1916
Epoch 4/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.6470 - loss: 0.1327 - val_accuracy: 0.5438 - val_loss: 0.2061
Epoch 5/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.6558 - loss: 0.1274 - val_accuracy: 0.6568 - val_loss: 0.1609
Epoch 6/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accuracy: 0.6762 - loss: 0.1186 - val_accuracy: 0.6855 - val_loss: 0.1656
Epoch 7/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.6815 - loss: 0.1157 - val_accuracy: 0.6121 - val_loss: 0.1558
Epoch 8/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accurac

In [13]:
process_week(2, best_params, results)


=== Processing Week 2 ===
Epoch 1/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 7s 5ms/step - accuracy: 0.5258 - loss: 0.1811 - val_accuracy: 0.6223 - val_loss: 0.2056
Epoch 2/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.6714 - loss: 0.1219 - val_accuracy: 0.6855 - val_loss: 0.1465
Epoch 3/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.7478 - loss: 0.0942 - val_accuracy: 0.6313 - val_loss: 0.1644
Epoch 4/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accuracy: 0.7943 - loss: 0.0757 - val_accuracy: 0.7276 - val_loss: 0.0743
Epoch 5/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.8312 - loss: 0.0598 - val_accuracy: 0.7468 - val_loss: 0.0982
Epoch 6/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accuracy: 0.8580 - loss: 0.0489 - val_accuracy: 0.7536 - val_loss: 0.0645
Epoch 7/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.8695 - loss: 0.0448 - val_accuracy: 0.8518 - val_loss: 0.0367
Epoch 8/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accurac

In [14]:
process_week(3, best_params, results)


=== Processing Week 3 ===
Epoch 1/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 7s 5ms/step - accuracy: 0.4657 - loss: 0.1992 - val_accuracy: 0.3986 - val_loss: 0.2712
Epoch 2/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.5918 - loss: 0.1530 - val_accuracy: 0.5587 - val_loss: 0.1589
Epoch 3/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.6823 - loss: 0.1136 - val_accuracy: 0.6543 - val_loss: 0.1298
Epoch 4/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.7369 - loss: 0.0914 - val_accuracy: 0.7084 - val_loss: 0.1066
Epoch 5/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accuracy: 0.7716 - loss: 0.0771 - val_accuracy: 0.8224 - val_loss: 0.0432
Epoch 6/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accuracy: 0.8062 - loss: 0.0635 - val_accuracy: 0.7378 - val_loss: 0.0608
Epoch 7/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.8241 - loss: 0.0549 - val_accuracy: 0.7548 - val_loss: 0.0597
Epoch 8/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accurac

In [15]:
process_week(4, best_params, results)


=== Processing Week 4 ===
Epoch 1/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - accuracy: 0.4502 - loss: 0.2028 - val_accuracy: 0.5493 - val_loss: 0.2298
Epoch 2/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.6497 - loss: 0.1274 - val_accuracy: 0.6146 - val_loss: 0.1981
Epoch 3/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.8035 - loss: 0.0639 - val_accuracy: 0.7501 - val_loss: 0.0718
Epoch 4/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.8983 - loss: 0.0306 - val_accuracy: 0.8558 - val_loss: 0.0317
Epoch 5/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.9162 - loss: 0.0222 - val_accuracy: 0.8566 - val_loss: 0.0275
Epoch 6/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accuracy: 0.9258 - loss: 0.0196 - val_accuracy: 0.9314 - val_loss: 0.0165
Epoch 7/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.9422 - loss: 0.0149 - val_accuracy: 0.8678 - val_loss: 0.0217
Epoch 8/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accurac

In [16]:
# Hiển thị dữ liệu của các tuần
print("\n=== Summary Results for All Weeks ===")
for result in results:
    print(f"Week {result['week']}:")
    print(f"  Train Time: {result['train_time']:.2f} seconds")
    print(f"  Test Time: {result['test_time']:.2f} seconds")
    print(f"  Accurancy: {result['accuracy']}")
    print(f"  Precision: {result['precision']}")
    print(f"  Recall: {result['recall']}")
    print(f"  F1-Score: {result['f1_score']}")
    print(f"  Macro Precision: {result['precision_macro']}")
    print(f"  Macro Recall: {result['recall_macro']}")
    print(f"  Macro F1-Score: {result['f1_macro']}")
    print(f"  Confusion Matrix:\n{result['confusion_matrix']}")
    print("\n=== AUC-ROC ===")
    print(f"AUC Macro: {result['auc_macro']:.4f}")
    print(f"AUC Weighted: {result['auc_weighted']:.4f}")
    print(f"AUC per Label: {result['auc_per_class']}")


=== Summary Results for All Weeks ===
Week 1:
  Train Time: 272.34 seconds
  Test Time: 0.37 seconds
  Accurancy: 0.8669920683343502
  Precision: [0.94324324 0.68627451 0.41441441 0.82258065 0.95717345]
  Recall: [0.93066667 0.63636364 0.89320388 0.49038462 0.89221557]
  F1-Score: [0.93691275 0.66037736 0.56615385 0.61445783 0.92355372]
  Macro Precision: 0.7647372520320687
  Macro Recall: 0.768566874154468
  Macro F1-Score: 0.740291101331166
  Confusion Matrix:
[[349  14   3   0   9]
 [ 19  35   1   0   0]
 [  0   0  92   1  10]
 [  0   0  32  51  21]
 [  2   2  94  10 894]]

=== AUC-ROC ===
AUC Macro: 0.9638
AUC Weighted: 0.9664
AUC per Label: [0.99056118 0.9746786  0.97026067 0.92214984 0.96117028]
Week 2:
  Train Time: 273.36 seconds
  Test Time: 0.39 seconds
  Accurancy: 0.8297742525930445
  Precision: [0.93305439 0.796875   0.9375     0.58715596 0.82563208]
  Recall: [0.59466667 0.92727273 0.72815534 0.61538462 0.94510978]
  F1-Score: [0.72638436 0.85714286 0.81967213 0.60093897